In [1]:
pip list


Package                      Version
---------------------------- -----------
absl-py                      2.2.2
annotated-types              0.7.0
appnope                      0.1.4
asttokens                    3.0.0
astunparse                   1.6.3
blis                         1.3.0
catalogue                    2.0.10
certifi                      2025.4.26
charset-normalizer           3.4.2
click                        8.2.0
cloudpathlib                 0.21.0
comm                         0.2.2
confection                   0.1.5
cymem                        2.0.11
debugpy                      1.8.14
decorator                    5.2.1
distlib                      0.3.9
en_core_web_sm               3.8.0
executing                    2.2.0
filelock                     3.18.0
flatbuffers                  25.2.10
fsspec                       2025.3.2
gast                         0.6.0
gensim                       4.3.3
google-pasta                 0.2.0
grpcio                       1.71

In [2]:
import gensim.downloader as api

# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
model = api.load("glove-wiki-gigaword-50")

[=================---------------------------------] 35.5% 23.4/66.0MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 66.0/66.0MB downloaded


In [3]:
model.most_similar([model['king']], topn=11)

[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839044332504272),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542160749435425),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492412328720093),
 ('ruler', 0.7434254288673401)]

In [5]:
pip install pandas


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/52/11/9eac327a38834f162b8250aab32a6781339c69afe7574368fffe46387edf/pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (89 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl.metadata
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/5c/23/c7abc0ca0a1526a0774eca151daeb8de62ec457e77262b66b359c3c7679e/tzdata-2025.2-py2.py3-none-any.whl.metadata
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py

In [6]:
import pandas as pd
from urllib import request

# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [17]:
print(songs_df[0:20])

                                                 title             artist
id                                                                       
0                         Gucci Time (w\/ Swizz Beatz)         Gucci Mane
1    Aston Martin Music (w\/ Drake & Chrisette Mich...          Rick Ross
2                        Get Back Up (w\/ Chris Brown)               T.I.
3                   Hot Toddy (w\/ Jay-Z & Ester Dean)              Usher
4                                         Whip My Hair             Willow
5                             Down On Me (w\/ 50 Cent)            Jeremih
6                                     Black And Yellow        Wiz Khalifa
7                                    Blowing Me Kisses         Soulja Boy
8                                          Lay It Down              Lloyd
9                        Good For My Money (w\/ Lloyd)          Baby Bash
10                                            Shake It       Elephant Man
11                                    

In [7]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [8]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# Extract recommendations
print_recommendations(2172)

,title,artist
id,,
2849,Run To The Hills,Iron Maiden
3167,Unchained,Van Halen
5586,The Last In Line,Dio
5634,Mr. Brownstone,Guns N' Roses
6624,Everybody Wants Some!!!,Van Halen
